In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}."
)

prompt = template.format(country_a="Mexico", country_b="Korea")


chat.predict(prompt)

The distance between Mexico and South Korea is approximately 8,900 miles (14,300 kilometers) when measured in a straight line.

'The distance between Mexico and South Korea is approximately 8,900 miles (14,300 kilometers) when measured in a straight line.'

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai","안녕하세요 제 이름은 {name}입니다"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language = "Korean",
    name = "Socrates",
    country_a = "Mexico",
    country_b = "Korea"
)

chat.predict_messages(prompt)

AIMessage(content='멕시코와 한국 사이의 거리는 대략 13,000km입니다. 제 이름은 Socreates입니다.')

In [28]:
from langchain.schema import BaseOutputParser

class CommandOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
p = CommandOutputParser()
p.parse("Hello, how, are, you")    

['Hello', 'how', 'are', 'you']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You ara a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items} in lower case. Do NOT reply with anything else."),
    ("human", "{question}"),
    
    ]
)

# prompt = template.format_messages(max_items=10, question="What are the Colors?")

# result = chat.predict_messages(prompt)

# p = CommandOutputParser()
# p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [ ]:
chain = template | chat | CommandOutputParser()

chain.invoke(
    {
        "max_items":5,
        "question":"What are the pokemons?"
    },
)

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [49]:
chef_template = ChatPromptTemplate.from_messages([
    ("system","You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients"),
    ("human","I want to cook {cuisine} food"),
])

chef_chain = chef_template | chat

In [50]:
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food, just say you don't know how to replace it"),
    ("human", "{recipe}"),
    
])
veg_chain = veg_chef_template | chat

final_chain = {"recipe": chef_chain} | veg_chain



In [51]:
final_chain.invoke({"cuisine": "indian"})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground turmeric
- 1 teaspoon garam masala
- 1 teaspoon ground coriander
- 1 teaspoon chili powder (adjust to taste)
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) crushed tomatoes
- 1 cup heavy cream
- Salt and pepper to taste
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, turmeric, garam masala, coriander, and chili powder. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for be

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use firm or extra-firm tofu for this recipe. Press the tofu to remove excess water and then cut it into bite-sized cubes.\n   - Instead of marinating the chicken, you can marinate the tofu in the yogurt and spice mixture for at least 30 minutes to allow the flavors to infuse.\n   - Bake the marinated tofu in the oven at 400°F (200°C) for about 20-25 minutes or until it develops a slight crispiness on the outside.\n\n2. **Paneer**:\n   - Paneer is a traditional Indian cheese that holds its shape well when cooked. You can cut the paneer into cubes and lightly pan-fry or bake them before adding to the sauce.\n   - You can also marinate the paneer cubes in the yogurt and spice mixture for added flavor before cooking.\n\nBy using tofu or paneer as a substitu

In [ ]:

examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

example_template = """
    Human: {question}
    AI: {answer}
"""
example_prompt = PromptTemplate.from_template("Human: {question}\n AI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)
chain = prompt | chat

chain.invoke({"country": "Korea"})

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')